In [ ]:
import sys
import os

# Use the latest version of pip.
!pip install --upgrade pip
# Install tfx and kfp Python packages.
!pip install --upgrade "tfx[kfp]<2"
!pip install python-dotenv

In [ ]:
# Git에서 제외된 .env 파일 생성 (pipeline directory)
"""
MYSQL_HOST = 11.333.11.222
MYSQL_PORT = 3306
MYSQL_DATABASE = advert_metadata
MYSQL_USERNAME = root
MYSQL_PASSWORD = abcdefg
"""

In [ ]:
# Read GCP project id from env.
shell_output=!gcloud config list --format 'value(core.project)' 2>/dev/null
GOOGLE_CLOUD_PROJECT=shell_output[0]
%env GOOGLE_CLOUD_PROJECT={GOOGLE_CLOUD_PROJECT}
print("GCP project ID:" + GOOGLE_CLOUD_PROJECT)

In [ ]:
"""
!cd foo, by itself, has no lasting effect, since the process with the changed directory immediately terminates.

%cd foo changes the current directory of the notebook process, which is a lasting effect.
"""

PIPELINE_NAME = 'advert_pipeline'
GIT_REPO_NAME = 'mlops-demo'
PROJECT_DIR=os.path.join(os.path.expanduser("~"), GIT_REPO_NAME, PIPELINE_NAME)

%cd {PROJECT_DIR}

In [ ]:
# Kubeflow endpoint
ENDPOINT='' # Enter your ENDPOINT here.
if not ENDPOINT:
    from absl import logging
    logging.error('Set your ENDPOINT in this cell.')

In [ ]:
# 파이프라인 생성 코드
!tfx pipeline create  --pipeline-path=kubeflow_runner.py --endpoint={ENDPOINT} \
--build-image

In [ ]:
# 파이프라인 run 돌리기
# kubeflow ui에서 생성된 파이프라인으로 만들어도 됨
!tfx run create --pipeline-name={PIPELINE_NAME} --endpoint={ENDPOINT}

In [ ]:
# 파이프라인 업데이트
!tfx pipeline update \
--pipeline-path=kubeflow_runner.py \
--endpoint={ENDPOINT}
# You can run the pipeline the same way.
# !tfx run create --pipeline-name {PIPELINE_NAME} --endpoint={ENDPOINT}